In [2]:
#imports
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import seaborn as sns

from PathFinder import find_path

import warnings
warnings.filterwarnings("ignore")

In [3]:
#read data
PATH = find_path()

#select files
#files = ['application_train.csv','application_test.csv',]
files = ['preprocessed_train.csv', 'preprocessed_test.csv']
"""         
          'bureau.csv', 
         'bureau_balance.csv', 'credit_card_balance.csv',
         'installments_payments.csv', 'POS_CASH_balance.csv',
         'previous_application.csv']
"""
data_files=[]
data=[]
print("files used: {}".format(files))

#actually read files
for file in files:
    stuff = pd.read_csv(PATH+file,header=0,engine='python')
    #print(stuff)
    data_files.append(file)
    data.append(stuff)

files used: ['preprocessed_train.csv', 'preprocessed_test.csv']


In [4]:
#select and convert columns
original_train = data[0] #save the original because we'll need some stuff later
original_test = data[1]
app_train = data[0].copy()
app_test = data[1].copy()

#delete ID cols
del app_train['SK_ID_CURR']
del app_test['SK_ID_CURR']

app_train_y = app_train['TARGET']
new_cols_x = app_train.columns.values.tolist()
new_cols_x.remove('TARGET')
app_train_x = app_train[new_cols_x]

print('Training Features shape: ', app_train_x.shape) #235 cols
print('Testing Features shape: ', app_test.shape) #235 cols

Training Features shape:  (307511, 235)
Testing Features shape:  (48744, 235)


In [17]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
# Create regularization penalty space
kernel= ['linear', 'poly', 'rbf', 'sigmoid']#, 'precomputed']
#penalty = ['l1', 'l2']
C = np.logspace(-1, 4, 20)
hyperparameters = dict(C=C, kernel=kernel)
svr = svm.SVR(max_iter=10)
clf = GridSearchCV(svr, hyperparameters, cv=5, verbose=0, scoring='roc_auc')
clf.fit(app_train_x,app_train_y)
clf.best_params_ 
#{'C': 1.0, 'kernel': 'poly'} #max_iter=1
#{'C': 263.6650898730358, 'kernel': 'rbf'} #max_iter=10, more C

{'C': 263.6650898730358, 'kernel': 'rbf'}

In [ ]:
from sklearn import svm
#svr = svm.SVR(gamma=.1,C=1)
#svr = svm.SVR(max_iter=1000, C=263.6650898730358, kernel='rbf')
#svr = svm.SVR(max_iter=20000, C=263.6650898730358, kernel='rbf')
svr = svm.SVR(max_iter=200000, C=263.6650898730358, kernel='rbf')
svr.fit(app_train_x,app_train_y)
svr_pred = svr.predict(app_test)
#svr_pred

In [ ]:
# Submission dataframe
submit = original_test[['SK_ID_CURR']]
submit['TARGET'] = svr_pred.clip(0, 1)

submit.head()

In [ ]:
# Save the submission to a csv file
overwrite = True
if overwrite:
    submit.to_csv('results/svr_2.csv', index = False)

Submitted to kaggle

svr_0 dont remember making
score: 0.51757  

svr_1 used gridsearch  
score: 0.57979